In [2]:
from IPython.display import display, Math, Latex

import pandas as pd
import numpy as np
import numpy_financial as npf
import yfinance as yf
import matplotlib.pyplot as plt
from datetime import datetime

## Group Assignment
### Team Number: [Number]
### Team Member Names: Arya Patel, [Name] & [Name]
### Team Strategy Chosen: RISKY

In [3]:
# Replace 'Tickers_Example.csv' with the actual ticker CSV file
ticker_file = 'Tickers_Example.csv'

# Read the CSV file into a pandas DataFrame with a single column
tickers_df = pd.read_csv(ticker_file, header=None, names=['Tickers'])

# Set the single column as the index
tickers_df.set_index('Tickers', inplace=True)

# Create an info cell for each ticker, saving time when filtering the tickers by volume and country.
info_column = {}
hist_column = {}
long_hist_column = {}
ticker_obj_lst = []

long_start_date = "2018-10-31"
start_date = "2023-01-01"
end_date = "2023-10-31"

tickers_df = tickers_df[~tickers_df.index.duplicated(keep='first')]
# Add info to dict
for ticker in tickers_df.index:
    yf_ticker = yf.Ticker(ticker)
    info_column[ticker] = yf_ticker.fast_info
    hist_column[ticker] = yf_ticker.history(start=start_date, end=end_date)
    long_hist_column[ticker] = yf_ticker.history(start=long_start_date, end=end_date)
    print('.', end='') # So we can tell if the code is running

# Create column for the info]
tickers_df['Info'] = pd.Series(info_column)
tickers_df['History'] = pd.Series(hist_column)
tickers_df['Long History'] = pd.Series(long_hist_column)
tickers_df

...

AGN: No timezone found, symbol may be delisted
AGN: No timezone found, symbol may be delisted


............

CELG: No timezone found, symbol may be delisted
CELG: No timezone found, symbol may be delisted


......

MON: No timezone found, symbol may be delisted
MON: No timezone found, symbol may be delisted


........

RTN: No timezone found, symbol may be delisted
RTN: No timezone found, symbol may be delisted


............

,Info,History,Long History
Tickers,,,
AAPL,"lazy-loading dict with keys = ['currency', 'da...",Open H...,Open H...
ABBV,"lazy-loading dict with keys = ['currency', 'da...",Open H...,Open H...
ABT,"lazy-loading dict with keys = ['currency', 'da...",Open H...,Open Hig...
ACN,"lazy-loading dict with keys = ['currency', 'da...",Open H...,Open H...
AGN,"lazy-loading dict with keys = ['currency', 'da...","Empty DataFrame Columns: [Open, High, Low, Clo...","Empty DataFrame Columns: [Open, High, Low, Clo..."
AIG,"lazy-loading dict with keys = ['currency', 'da...",Open Hig...,Open Hig...
AMZN,"lazy-loading dict with keys = ['currency', 'da...",Open H...,Open H...
AXP,"lazy-loading dict with keys = ['currency', 'da...",Open H...,Open H...
BA,"lazy-loading dict with keys = ['currency', 'da...",Open H...,Open H...


In [6]:
minVolume = 150000
allowed_currencies = ["USD", "CAD"]
threshold_day = 18
days_in_month = []

CAD_stocks = []
USD_stocks = []
all_stocks = []

for ticker in tickers_df.index:
    if tickers_df['History'][ticker].empty:
        print(ticker, "is delisted.")
    else:
        group_months = tickers_df["History"][ticker].reset_index().groupby(pd.Grouper(key="Date", freq='M'))
        dfs = [group for _, group in group_months]
        for i, df in enumerate(dfs):
            #recombine all the months with more trading days than set threshold
            if len(df.index) >= threshold_day:
                days_in_month.append(len(df.index))

        if tickers_df['Info'][ticker]['currency'] not in allowed_currencies:
            print(ticker, "is not in USD or CAD")
        elif tickers_df['History'][ticker]['Volume'].mean() * (np.mean(days_in_month)) < minVolume:
            print(ticker, "only has a volume of", tickers_df.iloc['Info'][ticker]['lastVolume'], "and average days of", np.mean(days_in_month))
        else:
            all_stocks.append(ticker)
            if tickers_df['Info'][ticker]['currency'] == 'USD':
                USD_stocks.append(ticker)
            elif tickers_df['Info'][ticker]['currency'] == 'CAD':
                CAD_stocks.append(ticker)
                                
print("\nValid CAD Stocks:\n",CAD_stocks)
print("\nValid USD Stocks:\n",USD_stocks)
print("\nValid All Valid Stocks:\n", all_stocks)

AGN is delisted.
CELG is delisted.
MON is delisted.
RTN is delisted.

Valid CAD Stocks:
 ['RY.TO', 'SHOP.TO', 'T.TO', 'TD.TO']

Valid USD Stocks:
 ['AAPL', 'ABBV', 'ABT', 'ACN', 'AIG', 'AMZN', 'AXP', 'BA', 'BAC', 'BIIB', 'BK', 'BLK', 'BMY', 'C', 'CAT', 'CL', 'KO', 'LLY', 'LMT', 'MO', 'MRK', 'PEP', 'PFE', 'PG', 'PM', 'PYPL', 'QCOM', 'TXN', 'UNH', 'UNP', 'UPS', 'USB', 'YCBD']

Valid All Valid Stocks:
 ['AAPL', 'ABBV', 'ABT', 'ACN', 'AIG', 'AMZN', 'AXP', 'BA', 'BAC', 'BIIB', 'BK', 'BLK', 'BMY', 'C', 'CAT', 'CL', 'KO', 'LLY', 'LMT', 'MO', 'MRK', 'PEP', 'PFE', 'PG', 'PM', 'PYPL', 'QCOM', 'RY.TO', 'SHOP.TO', 'T.TO', 'TD.TO', 'TXN', 'UNH', 'UNP', 'UPS', 'USB', 'YCBD']


In [7]:
stock_stds = []
stocks_df = pd.DataFrame()

##Retreiving CADUSD currency conversion data
exchange = yf.Ticker('CADUSD=x')
exchange_hist = exchange.history(start=start_date, end=end_date)

exchange_hist.index = pd.to_datetime(exchange_hist.index)
exchange_hist.index = exchange_hist.index.date

##Debugging
# print(tickers_df['History']['AAPL']['Close'].iloc[0])
# print(exchange_hist.Close.iloc[1])
# print(tickers_df['History']['AAPL']['Close'].iloc[0] * (1/exchange_hist.Close.iloc[1]))

##Creating dataframe for all stocks closing prices in CAD
for ticker in all_stocks:
    currCurrency = tickers_df['Info'][ticker]['currency']
    currClose = tickers_df['History'][ticker]['Close']
    
    ##changing stock index to match with CADUSD conversion index
    currClose.index = pd.to_datetime(currClose.index)
    currClose.index = currClose.index.date
    
    ##make sure all stocks are priced in CAD
    if currCurrency == "USD":                                       
        stocks_df[ticker] = currClose * (1/exchange_hist.Close)
    else:
        stocks_df[ticker] = currClose
        
    ##Making list of 
    stocks_df = stocks_df.dropna()
    
stocks_df
##reason to keep all stocks in the same currency is to increase accuracy of the correlation calculations

,AAPL,ABBV,ABT,ACN,AIG,AMZN,AXP,BA,BAC,BIIB,...,RY.TO,SHOP.TO,T.TO,TD.TO,TXN,UNH,UNP,UPS,USB,YCBD
2023-01-03,168.639803,211.621955,145.665504,360.714564,83.711194,116.363344,196.572718,264.929315,44.415907,369.659044,...,122.739281,48.790001,25.214359,84.689217,214.563552,695.157322,276.074091,228.572671,57.969776,14.094582
2023-01-04,171.825503,215.140095,149.087180,362.538230,85.669408,116.421281,202.849784,278.459358,45.635063,370.308280,...,123.832169,50.610001,25.520916,85.790459,224.286735,681.944918,280.644092,232.919637,60.282250,14.768028
2023-01-05,167.653624,211.907287,146.484490,349.084947,84.022264,112.088154,195.258901,276.431070,44.912033,366.241823,...,123.467865,48.830002,25.492178,84.384224,218.257955,653.136789,268.617178,225.459282,58.984102,14.321176
2023-01-06,174.786467,217.070924,149.331011,359.337031,85.871867,116.723626,201.356783,288.825880,45.611842,378.660220,...,125.040092,49.560001,25.884954,84.345154,230.288889,656.813348,281.984519,233.373985,60.142450,14.522653
2023-01-09,173.887177,208.759965,147.720630,362.036709,84.185858,117.369912,199.810640,280.217986,44.509437,369.091829,...,125.970009,49.810001,25.827475,84.081490,230.151097,650.852846,278.666530,234.768279,59.975385,14.147266
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-10-24,237.100988,200.276445,129.780668,405.302802,81.953050,175.979357,197.689320,249.623493,34.864609,345.100783,...,109.029999,71.870003,22.360001,76.830002,199.284689,718.646268,281.216554,202.030681,42.968202,1.314096
2023-10-25,234.809448,199.610478,128.580154,402.189148,83.768794,166.809285,197.219453,244.229456,35.109788,339.032764,...,109.110001,66.900002,22.280001,76.919998,193.072336,728.593420,282.005123,199.568144,42.997469,1.266976
2023-10-26,230.053533,200.418114,129.719665,403.099924,83.990649,165.041283,197.850774,247.196144,36.053178,332.760333,...,110.209999,64.529999,22.320000,77.410004,196.970043,729.290041,279.149871,188.561638,43.851816,1.311276
2023-10-27,232.073362,191.917889,128.262986,400.661261,82.234739,176.460030,195.205627,248.223765,34.769838,323.965929,...,108.470001,64.349998,22.010000,76.160004,195.910134,724.765275,278.656005,184.098169,42.326094,1.257074


In [14]:
##std-dataframe function

def stds(dataframe):
    
    ##define variables
    stock_stds = []
    stock_list = sorted(dataframe)
    
    ##append each stock's percent std to a list
    for stock in all_stocks:
        currPctChange = dataframe[stock].pct_change() * 100
        stock_stds.append(currPctChange.std())
        
    ##make dataframe with stock names as indexes with their associated std vlaue   
    stock_stds = pd.DataFrame(stock_stds, columns=['stds'])
    stock_stds.index = stock_list
    
    return stock_stds
    
    
corr_d = stds(stocks_df)
corr_d

,stds
AAPL,1.340555
ABBV,1.339442
ABT,1.310785
ACN,1.503597
AIG,1.798248
AMZN,2.207021
AXP,1.714515
BA,1.801327
BAC,1.666901
BIIB,1.344842


In [9]:
marketTicker = yf.Ticker("^GSPC")
marketHist = marketTicker.history(start=long_start_date, end=end_date)

beta = {}

def beta_volatility(ticker):
    prices = pd.DataFrame(tickers_df["Long History"][ticker]["Close"])
    prices.columns = [ticker]
    prices["Market"] = marketHist['Close']

    monthly_change = prices.resample("M").ffill().pct_change()
    monthly_change = monthly_change.drop(index=monthly_change.index[0])

    return (monthly_change.cov()/monthly_change["Market"].var()).iat[0, 1]


for ticker in (CAD_stocks + USD_stocks):
    beta[ticker] = beta_volatility(ticker)

sorted(beta.items(), key=lambda x:x[1], reverse=True)


C:\Users\matth\AppData\Local\Temp\ipykernel_10260\670029220.py:11: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Call ffill before calling pct_change to retain current behavior and silence this warning.
  monthly_change = prices.resample("M").ffill().pct_change()
C:\Users\matth\AppData\Local\Temp\ipykernel_10260\670029220.py:11: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Call ffill before calling pct_change to retain current behavior and silence this warning.
  monthly_change = prices.resample("M").ffill().pct_change()
C:\Users\matth\AppData\Local\Temp\ipykernel_10260\670029220.py:11: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Call ffill before calling pct_change to retain current behavior and silence this warning.
  monthly_change = prices.resample(

[('YCBD', 2.1401685163092337),
 ('SHOP.TO', 1.8105683661552798),
 ('C', 1.5566631536079352),
 ('BA', 1.4762122007970342),
 ('BAC', 1.3851007198935334),
 ('PYPL', 1.379281690066955),
 ('AAPL', 1.3036338892642167),
 ('BLK', 1.283591100945253),
 ('ACN', 1.230450673430567),
 ('QCOM', 1.2167441584332797),
 ('AXP', 1.2146618158649238),
 ('AMZN', 1.1719942897819302),
 ('UPS', 1.1049470026976496),
 ('UNP', 1.1025178809560923),
 ('BK', 1.0953385838889629),
 ('CAT', 1.087944180384716),
 ('AIG', 1.035101685932042),
 ('USB', 1.0267306462168988),
 ('TXN', 1.016828366563042),
 ('PM', 0.7766933001995219),
 ('ABT', 0.6933939700031805),
 ('MO', 0.6535224653145476),
 ('TD.TO', 0.6386755561701395),
 ('UNH', 0.632112953336177),
 ('PFE', 0.6305324963138514),
 ('LMT', 0.5987249741149856),
 ('KO', 0.5908109682170692),
 ('PEP', 0.572641913495487),
 ('RY.TO', 0.5664455971567733),
 ('ABBV', 0.507022396490688),
 ('PG', 0.4747911609412613),
 ('CL', 0.4606669209449301),
 ('T.TO', 0.4596694417486112),
 ('BMY', 0.38

In [16]:
combined_risk = beta

for ticker in combined_risk.keys():
    combined_risk[ticker] += corr_d["stds"][ticker]

sorted(combined_risk.items(), key=lambda x:x[1], reverse=True)


[('YCBD', 15.258183198022778),
 ('SHOP.TO', 8.921438733957544),
 ('USB', 6.208743039609542),
 ('PYPL', 5.925784080954843),
 ('AMZN', 5.586035528039014),
 ('QCOM', 5.395363524468188),
 ('BA', 5.078866663411475),
 ('C', 4.8207302123869304),
 ('BAC', 4.7189025048928155),
 ('CAT', 4.683777653350453),
 ('AXP', 4.643691416241061),
 ('AIG', 4.631598283566004),
 ('UNP', 4.37366758423113),
 ('BK', 4.3613111385038374),
 ('ACN', 4.237644547722475),
 ('TXN', 4.165586782959859),
 ('UPS', 4.160713451462888),
 ('LLY', 4.142828070160359),
 ('BLK', 3.9904546832949386),
 ('AAPL', 3.984743653669427),
 ('UNH', 3.6179796688793804),
 ('PFE', 3.466369132675372),
 ('ABT', 3.3149647563708404),
 ('ABBV', 3.1859055787924335),
 ('LMT', 3.05229971366784),
 ('PM', 3.0083023009493504),
 ('MO', 2.9725613081605733),
 ('MRK', 2.963344819630598),
 ('BIIB', 2.789405936618034),
 ('TD.TO', 2.7880261700561952),
 ('BMY', 2.781069283389649),
 ('CL', 2.748720905297959),
 ('PEP', 2.599411589211738),
 ('T.TO', 2.549241670665106)

## Contribution Declaration

The following team members made a meaningful contribution to this assignment:

Insert Names Here.